In [68]:
# Dependencies
import requests
import json
import pandas as pd

from sqlalchemy import create_engine
# Google developer API key
from config import gkey

In [69]:
accidents_csv = "../../Resources/raw/Saferparks-dataset-legacy-v2.csv"
table_name = "accident_data"

geocode_json_base = "https://maps.googleapis.com/maps/api/geocode/json?"

In [70]:
us_state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [71]:
df = pd.read_csv(accidents_csv, encoding="ANSI")

In [72]:
df["acc_state"].replace(us_state_abbrev, inplace=True)
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
0,1005755,9/12/2009,Wisconsin,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,NaN,"Rider was riding down slide, stopped, stood up...","chin, ribs",NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,NaN
1,1005754,6/28/2009,Wisconsin,Madison,P,Media Report,Unknown,amusement ride,spinning,Ferris/gondola wheel,...,NaN,NaN,The drive cable of the Ferris wheel came off i...,Minor injuries,NaN,Equipment failure,1.0,1.0,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html
2,1005753,5/30/2009,Wisconsin,Wisconsin Dells,F,Media Report,Amusement park,recreation,go-kart,Go-kart,...,8.0,NaN,Parent reports her son's kart was repeatedly h...,"Fracture, bruising, lacerations",NaN,Collision: patron-controlled vehicles,NaN,NaN,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html
3,1004578,4/17/2009,Washington,Puyallup,P,Washington Dept. of Labor and Industries,Carnival or rental,amusement ride,spinning,Swing ride,...,NaN,NaN,Swing tipped over on its side during operation...,"6 children were transported to the hospital, t...",NaN,Equipment failure,1.0,NaN,NaN,http://www.king5.com/article/news/local/amusem...
4,1007272,4/5/2009,California,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Track ride,...,44.0,F,Patron's attorney stated that she had a right ...,Right vertegral artery tear,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN


In [76]:
df["acc_date"] = pd.to_datetime(df["acc_date"], format='%m/%d/%Y')

In [77]:
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
0,1005755,2009-09-12,Wisconsin,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,NaN,"Rider was riding down slide, stopped, stood up...","chin, ribs",NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,NaN
1,1005754,2009-06-28,Wisconsin,Madison,P,Media Report,Unknown,amusement ride,spinning,Ferris/gondola wheel,...,NaN,NaN,The drive cable of the Ferris wheel came off i...,Minor injuries,NaN,Equipment failure,1.0,1.0,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html
2,1005753,2009-05-30,Wisconsin,Wisconsin Dells,F,Media Report,Amusement park,recreation,go-kart,Go-kart,...,8.0,NaN,Parent reports her son's kart was repeatedly h...,"Fracture, bruising, lacerations",NaN,Collision: patron-controlled vehicles,NaN,NaN,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html
3,1004578,2009-04-17,Washington,Puyallup,P,Washington Dept. of Labor and Industries,Carnival or rental,amusement ride,spinning,Swing ride,...,NaN,NaN,Swing tipped over on its side during operation...,"6 children were transported to the hospital, t...",NaN,Equipment failure,1.0,NaN,NaN,http://www.king5.com/article/news/local/amusem...
4,1007272,2009-04-05,California,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Track ride,...,44.0,F,Patron's attorney stated that she had a right ...,Right vertegral artery tear,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN


In [78]:
unique_cities = df["acc_city"].unique()

city_state_df = df[["acc_state", "acc_city"]].loc[df["acc_city"].isin(unique_cities)]
city_state_df.drop_duplicates(inplace=True)
city_state_df.reset_index(inplace=True, drop=True)
city_state_df.head()

,acc_state,acc_city
0,Wisconsin,Green Bay
1,Wisconsin,Madison
2,Wisconsin,Wisconsin Dells
3,Washington,Puyallup
4,California,Anaheim


In [79]:

city_lat = []
city_lng = []
for i in range(len(city_state_df)):
    params = {
        "address": f'{city_state_df.loc[i,"acc_city"]}, {city_state_df.loc[i,"acc_state"]}',
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        city_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        city_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        
        print(f'{params["address"]} : {i}')
    except:
        print(f'Address: {params["address"]} not found : {i}')
        city_lat.append("null")
        city_lng.append("null") 


Green Bay, Wisconsin : 0
Madison, Wisconsin : 1
Wisconsin Dells, Wisconsin : 2
Puyallup, Washington : 3
Anaheim, California : 4
Naples, Florida : 5
Buena Park, California : 6
Los Angeles, California : 7
Riverside, California : 8
Upland, California : 9
San Dimas, California : 10
San Diego, California : 11
Irvine, California : 12
Palm Springs, California : 13
Phillips, Wisconsin : 14
West Allis, Wisconsin : 15
Lake Gregory, California : 16
Escondido, California : 17
San Jose, California : 18
Carlsbad, California : 19
Milwaukee, Wisconsin : 20
Vista, California : 21
Santa Barbara, California : 22
Angels Camp, California : 23
Universal City, California : 24
Aurora, Colorado : 25
Santa Monica, California : 26
nan, Florida : 27
Egg Harbor Twp., New Jersey : 28
Orlando, Florida : 29
Dania Beach, Florida : 30
Stuttgart, Arkansas : 31
Cherry Hill, New Jersey : 32
nan, Illinois : 33
Jacksonville, Florida : 34
Santa Clara, California : 35
nan, Kentucky : 36
nan, Oklahoma : 37
Jackson, New Jersey 

In [80]:
city_df = pd.DataFrame({
    "acc_city": city_state_df["acc_city"],
    "acc_state": city_state_df["acc_state"],
    "city_lat": city_lat,
    "city_lng": city_lng
})
city_df = city_df[city_df.acc_city != "null"]
city_df.dropna(inplace=True)
city_df.head()

,acc_city,acc_state,city_lat,city_lng
0,Green Bay,Wisconsin,44.513319,-88.013296
1,Madison,Wisconsin,43.073052,-89.401230
2,Wisconsin Dells,Wisconsin,43.627479,-89.770958
3,Puyallup,Washington,47.185378,-122.292897
4,Anaheim,California,33.836593,-117.914301


In [81]:
acc_states = df["acc_state"].unique()
state_lat = []
state_lng = []
for state in acc_states:
    params = {
        "address": state,
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        state_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        state_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        print(state)
    except:
        print(f'address: {city} not found')
        state_lat.append("null")
        state_lng.append("null") 

Wisconsin
Washington
California
Florida
Colorado
New Jersey
Arkansas
Illinois
Kentucky
Oklahoma
Texas
Ohio
Pennsylvania
New York
Missouri
Nevada
Massachusetts
Georgia
Iowa
Virginia
Maryland
Michigan
West Virginia
North Carolina
Tennessee
Arizona
Connecticut
Indiana
Rhode Island
Louisiana


In [82]:
state_df = pd.DataFrame({
    "acc_state": acc_states,
    "state_lat": state_lat,
    "state_lng": state_lng
})

state_df.head()

,acc_state,state_lat,state_lng
0,Wisconsin,43.784440,-88.787868
1,Washington,47.751074,-120.740139
2,California,36.778261,-119.417932
3,Florida,27.664827,-81.515754
4,Colorado,39.550051,-105.782067


In [83]:
df = pd.merge(df, city_df, how="left", on=["acc_city", "acc_state"])
df = pd.merge(df, state_df, how="left", on="acc_state")

In [84]:
df.drop_duplicates(subset="acc_id", inplace=True)
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,report,category,mechanical,op_error,employee,notes,city_lat,city_lng,state_lat,state_lng
0,1005755,2009-09-12,Wisconsin,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,NaN,44.513319,-88.013296,43.784440,-88.787868
1,1005754,2009-06-28,Wisconsin,Madison,P,Media Report,Unknown,amusement ride,spinning,Ferris/gondola wheel,...,NaN,Equipment failure,1.0,1.0,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html,43.073052,-89.401230,43.784440,-88.787868
2,1005753,2009-05-30,Wisconsin,Wisconsin Dells,F,Media Report,Amusement park,recreation,go-kart,Go-kart,...,NaN,Collision: patron-controlled vehicles,NaN,NaN,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html,43.627479,-89.770958,43.784440,-88.787868
3,1004578,2009-04-17,Washington,Puyallup,P,Washington Dept. of Labor and Industries,Carnival or rental,amusement ride,spinning,Swing ride,...,NaN,Equipment failure,1.0,NaN,NaN,http://www.king5.com/article/news/local/amusem...,47.185378,-122.292897,47.751074,-120.740139
4,1007272,2009-04-05,California,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Track ride,...,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN,33.836593,-117.914301,36.778261,-119.417932


In [85]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../../Resources/park_accidents.sqlite", encoding="ANSI")
conn = engine.connect()

In [86]:
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

In [87]:
state_count = df["acc_state"].value_counts().keys()

In [88]:
state = df["acc_state"].value_counts().keys()
count = df["acc_state"].value_counts().values

state_count_df = pd.DataFrame({
    "acc_state": state,
    "count": count
})

state_count_df.head()

,acc_state,count
0,New Jersey,5648
1,California,3430
2,Texas,3322
3,Florida,1074
4,Pennsylvania,270


In [89]:
state_count_df = pd.merge(state_count_df, state_df, how="outer", on="acc_state")
state_count_df

,acc_state,count,state_lat,state_lng
0,New Jersey,5648,40.058324,-74.405661
1,California,3430,36.778261,-119.417932
2,Texas,3322,31.968599,-99.901813
3,Florida,1074,27.664827,-81.515754
4,Pennsylvania,270,41.203322,-77.194525
5,Michigan,268,44.314844,-85.602364
6,Illinois,155,40.633125,-89.398528
7,Oklahoma,154,35.467560,-97.516428
8,Wisconsin,117,43.784440,-88.787868
9,Colorado,107,39.550051,-105.782067


In [90]:
state_count_df.to_csv("../../Resources/raw/state_counts.csv", index=False)

# TESTING 

In [91]:
state_counts_csv = '../../Resources/raw/state_counts.csv'
csv_df = pd.read_csv(state_counts_csv)

In [92]:
csv_df.head()

,acc_state,count,state_lat,state_lng
0,NJ,5648,40.058324,-74.405661
1,CA,3430,36.778261,-119.417932
2,TX,3322,31.968599,-99.901813
3,FL,1074,27.664827,-81.515754
4,PA,270,41.203322,-77.194525


In [102]:
for index, row in csv_df.iterrows():
    print(row["acc_state"], row["state_lat"])

NJ 40.0583238
CA 36.778261
TX 31.968598800000002
FL 27.664827399999997
PA 41.2033216
MI 44.3148443
IL 40.6331249
OK 35.007751899999995
WI 43.78443970000001
CO 37.06250000000001
OH 40.4172871
NY 40.7127753
MD 39.0457549
KY 37.839333200000006
NV 38.8026097
IA 41.8780025
NC 35.759573100000004
MA 42.4072107
WV 38.59762620000001
CT 41.60322070000001
MO 37.9642529
WA 47.751074100000004
IN 40.2671941
RI 41.5800945
AR 36.1481727
VA 36.103255
TN 35.517491299999996
GA 32.1656221
LA 34.0522342
AZ 34.048928100000005


In [108]:
    params = {
        "address": "VA",
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()

In [109]:
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "9322",
                    "short_name": "9322",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "East 41st Street",
                    "short_name": "E 41st St",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "Tulsa",
                    "short_name": "Tulsa",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Tulsa County",
                    "short_name": "Tulsa County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
              

In [67]:
df["bus_type"].unique()

array(['Amusement park', 'Unknown', 'Carnival or rental',
       'Family entertainment center', 'Water park', 'Pool waterslide',
       'Zoo or museum', 'Mall, store or restaurant', 'Other',
       'Sports or recreation facility', 'City or county park'],
      dtype=object)